In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.optimizers import Adam

## Read Data

In [2]:
# Load the data
train_data = pd.read_csv('train_sample.csv')
test_data = pd.read_csv('test_sample.csv')

In [3]:
# Prepare the data
X = train_data[['feature_0', 'feature_1', 'feature_2']].values
y = train_data['target'].values
X_test = test_data[['feature_0', 'feature_1', 'feature_2']].values

In [4]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Parameters
n_features = X_train.shape[1]
latent_factors = 10
batch_size = 32
n_epochs = 100

## Finetuning

In [11]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

# More complex model
inputs = [Input(shape=(1,)) for _ in range(n_features)]
embeddings = [Embedding(input_dim=np.max(X_train[:, i])+1, output_dim=20)(inputs[i]) for i in range(n_features)]
flat_embeddings = [Flatten()(embed) for embed in embeddings]
concat = concatenate(flat_embeddings)
dense = Dense(50, activation='relu', kernel_regularizer=l2(0.01))(concat)
dropout = Dropout(0.5)(dense)
outputs = Dense(1)(dropout)
model = Model(inputs, outputs)

# Optimizer with a different learning rate
optimizer = Adam(clipnorm=1.0, learning_rate=0.001)

model.compile(optimizer=optimizer, loss='mse')

# Define the early stopping callback
earlystopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [12]:
# Fit the model with early stopping
model.fit([X_train[:, i] for i in range(n_features)], y_train, epochs=n_epochs, batch_size=batch_size, verbose=1,
          validation_data=([X_val[:, i] for i in range(n_features)], y_val), callbacks=[earlystopper])

Epoch 1/100
225/225 [==============================] - 2s 4ms/step - loss: 0.6163 - val_loss: 0.2199
Epoch 2/100
225/225 [==============================] - 1s 3ms/step - loss: 0.2037 - val_loss: 0.1539
Epoch 3/100
225/225 [==============================] - 1s 4ms/step - loss: 0.1713 - val_loss: 0.1383
Epoch 4/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1608 - val_loss: 0.1325
Epoch 5/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1504 - val_loss: 0.1275
Epoch 6/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1506 - val_loss: 0.1281
Epoch 7/100
225/225 [==============================] - 1s 4ms/step - loss: 0.1417 - val_loss: 0.1240
Epoch 8/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1426 - val_loss: 0.1255
Epoch 9/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1406 - val_loss: 0.1241
Epoch 10/100
225/225 [==============================] - 1s 3ms/step - loss: 0.1412 - val_lo

In [13]:
# Refit the model to the entire dataset
best_epoch = earlystopper.stopped_epoch - 1 if earlystopper.stopped_epoch > 0 else n_epochs
model.fit([X[:, i] for i in range(n_features)], y, epochs=best_epoch, batch_size=batch_size, verbose=1)

Epoch 1/15
282/282 [==============================] - 1s 4ms/step - loss: 0.1439
Epoch 2/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1404
Epoch 3/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1397
Epoch 4/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1414
Epoch 5/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1354
Epoch 6/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1376
Epoch 7/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1381
Epoch 8/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1382
Epoch 9/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1366
Epoch 10/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1361
Epoch 11/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1386
Epoch 12/15
282/282 [==============================] - 1s 3ms/step - loss: 0.1418
Epoch 13/15
282/282 [====

### v3

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from math import sqrt

# Function to create and compile a model
def create_model(latent_dim, clipnorm, reg_lambda):
    inputs = [Input(shape=(1,)) for _ in range(n_features)]
    embeddings = [Embedding(input_dim=np.max(X_train[:, i])+1, output_dim=latent_dim, embeddings_regularizer=l2(reg_lambda))(inputs[i]) for i in range(n_features)]
    flat_embeddings = [Flatten()(embed) for embed in embeddings]
    concat = concatenate(flat_embeddings)
    dense1 = Dense(100, activation='relu')(concat)
    batch_norm1 = BatchNormalization()(dense1)
    # dropout1 = Dropout(0.3)(batch_norm1)
    dense2 = Dense(50, activation='relu')(batch_norm1)
    batch_norm2 = BatchNormalization()(dense2)
    # dropout2 = Dropout(0.2)(batch_norm2)
    outputs = Dense(1)(batch_norm2)
    model = Model(inputs, outputs)

    # Compile model with a refined optimizer
    optimizer = Adam(learning_rate=0.001, clipnorm=clipnorm)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Load data here (omitted for brevity, assume X_train, y_train, X_test are loaded and prepared)
# Assume n_features is set based on your dataset

# Grid of hyperparameters
latent_dims = [3, 5]
clipnorms = [0.5, 1.0, 1.5]
reg_lambdas = [0.0001, 0.0005, 0.0008]

best_score = float('inf')
best_model = None

# K-fold cross-validation
kf = KFold(n_splits=5)

for train_index, val_index in kf.split(X_train):
    X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
    y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
    
    for latent_dim in latent_dims:
        for clipnorm in clipnorms:
            for reg_lambda in reg_lambdas:
                print(f"Training model with {latent_dim} latent factors, clipnorm={clipnorm}, L2 reg={reg_lambda}")
                model = create_model(latent_dim, clipnorm, reg_lambda)
                early_stopper = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
                model.fit([X_train_kf[:, i] for i in range(n_features)], y_train_kf, epochs=50, batch_size=32, verbose=0,
                          validation_data=([X_val_kf[:, i] for i in range(n_features)], y_val_kf), callbacks=[early_stopper])
                val_loss = model.evaluate([X_val_kf[:, i] for i in range(n_features)], y_val_kf, verbose=0)
                val_rmse = sqrt(val_loss)
                print(f"Validation RMSE: {val_rmse:.4f}")
                
                if val_loss < best_score:
                    best_score = val_loss
                    best_model = model
                    # Save best model
                    model.save('best_model', save_format='tf')

Training model with 3 latent factors, clipnorm=0.5, L2 reg=0.0001
Validation RMSE: 0.3296
INFO:tensorflow:Assets written to: best_model\assets
Training model with 3 latent factors, clipnorm=0.5, L2 reg=0.0005
Validation RMSE: 0.2644
INFO:tensorflow:Assets written to: best_model\assets
Training model with 3 latent factors, clipnorm=0.5, L2 reg=0.0008
Validation RMSE: 0.3386
Training model with 3 latent factors, clipnorm=1.0, L2 reg=0.0001
Validation RMSE: 0.2646
Training model with 3 latent factors, clipnorm=1.0, L2 reg=0.0005
Validation RMSE: 0.2755
Training model with 3 latent factors, clipnorm=1.0, L2 reg=0.0008
Validation RMSE: 0.2628
INFO:tensorflow:Assets written to: best_model\assets
Training model with 3 latent factors, clipnorm=1.5, L2 reg=0.0001
Validation RMSE: 0.2720
Training model with 3 latent factors, clipnorm=1.5, L2 reg=0.0005
Validation RMSE: 0.2628
INFO:tensorflow:Assets written to: best_model\assets
Training model with 3 latent factors, clipnorm=1.5, L2 reg=0.0008
Va


KeyboardInterrupt



In [54]:
def create_model(latent_dim, clipnorm, reg_lambda):
    inputs = [Input(shape=(1,)) for _ in range(n_features)]
    embeddings = [Embedding(input_dim=np.max(X_train[:, i])+1, output_dim=latent_dim, embeddings_regularizer=l2(reg_lambda))(inputs[i]) for i in range(n_features)]
    flat_embeddings = [Flatten()(embed) for embed in embeddings]
    concat = concatenate(flat_embeddings)
    dense1 = Dense(200, activation='relu')(concat)
    batch_norm1 = BatchNormalization()(dense1)
    dropout1 = Dropout(0.3)(batch_norm1)
    dense2 = Dense(100, activation='relu')(dropout1)
    batch_norm2 = BatchNormalization()(dense2)
    dropout2 = Dropout(0.2)(batch_norm2)
    outputs = Dense(1)(dropout2)
    model = Model(inputs, outputs)

    # Compile model with a refined optimizer
    optimizer = Adam(learning_rate=0.001, clipnorm=clipnorm)
    model.compile(optimizer=optimizer, loss='mse')
    return model

In [55]:
model = create_model(5, 1.5, 0.0001)

In [56]:
model.fit([X_train_kf[:, i] for i in range(n_features)], y_train_kf, epochs=300, batch_size=128, verbose=0,
validation_data=([X_val_kf[:, i] for i in range(n_features)], y_val_kf), callbacks=[early_stopper])
val_loss = model.evaluate([X_val_kf[:, i] for i in range(n_features)], y_val_kf, verbose=0)
val_rmse = sqrt(val_loss)
print(f"Validation RMSE: {val_rmse:.4f}")

Validation RMSE: 0.2311


In [57]:
# Predict and average predictions if multiple folds/models are used
predictions = model.predict([X_test[:, i] for i in range(n_features)])
# Implement averaging if using multiple models from different folds or configurations

# Save predictions
sub = pd.DataFrame({'ID': test_data['ID'], 'target': predictions.flatten()})
sub.to_csv('submission3.csv', index=False)

32/32 [==============================] - 0s 2ms/step
